# Proyecto: Recomendación de Planes Móviles para Megaline

## Descripción del proyecto

La compañía móvil Megaline no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra.

En este proyecto, se utilizarán datos de comportamiento de suscriptores que ya se han cambiado a los planes nuevos. El objetivo es crear un modelo de clasificación con una exactitud ("accuracy") de al menos 0.75.

### Tabla de contenidos

1.- Exploración de datos

2.- Segmentación de datos (Entrenamiento, Validación, Prueba)

3.- Investigación de modelos (Árbol de Decisión, Bosque Aleatorio, Regresión Logística)

4.- Evaluación final con el conjunto de prueba

5.- Prueba de cordura

## 1. Exploración de datos

Cargamos el dataset y revisamos la información general para asegurarnos de que los tipos de datos sean correctos y no haya valores nulos.

In [2]:
# Importamos librerias:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [3]:
# Cargamos el DataSet para su revision

df = pd.read_csv("/datasets/users_behavior.csv")

# Mostramos la informacion del DS, asi como las primeras y ultimas filas

df.info()
display(df.head())
display(df.tail())

# Verificar el equilibrio de clases
print("\nDistribución de clases (is_ultra):")
print(df['is_ultra'].value_counts(normalize=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1



Distribución de clases (is_ultra):
0    0.693528
1    0.306472
Name: is_ultra, dtype: float64


**Conclusion**

El conjunto de datos contiene 3,214 registros sin valores ausentes ni tipos de datos incorrectos, lo que facilita el preprocesamiento. 

Sin embargo, se observó un desequilibrio de clases: aproximadamente el 69% de los usuarios utilizan el plan "Smart" y el 31% el plan "Ultra".

Este desequilibrio debe tenerse en cuenta, ya que un modelo que simplemente "adivine" la clase mayoritaria ya tendría una exactitud base cercana al 70%.


## 2. Segmentar los datos fuente

Para evaluar correctamente los modelos, dividiremos los datos en tres conjuntos:

• **Entrenamiento (60%):** Para entrenar el modelo.

• **Validación (20%):** Para ajustar los hiperparámetros y elegir el mejor modelo.

• **Prueba (20%):** Para la evaluación final.

In [4]:
# Primero, dividimos en (Entrenamiento + Validación) y (Prueba)
# 80% para train_val y 20% para test
df_train_val, df_test = train_test_split(df, test_size=0.20, random_state=12345)

# Luego, dividimos (Entrenamiento + Validación) en (Entrenamiento) y (Validación)
# De ese 80%, tomamos 25% para validación (que es el 20% del total original)
df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=12345)

# Definir features (características) y target (objetivo)
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_val = df_val.drop(['is_ultra'], axis=1)
target_val = df_val['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

print(f"Tamaño Entrenamiento: {len(df_train)}")
print(f"Tamaño Validación:    {len(df_val)}")
print(f"Tamaño Prueba:        {len(df_test)}")

Tamaño Entrenamiento: 1928
Tamaño Validación:    643
Tamaño Prueba:        643


**Conclusion**
La división de los datos en conjuntos de Entrenamiento (60%), Validación (20%) y Prueba (20%) se realizó correctamente. 

Esta estrategia es crucial para evitar el sobreajuste (overfitting): entrenamos con el primer conjunto, ajustamos la complejidad del modelo con el segundo y, finalmente, obtenemos una métrica de rendimiento realista con el tercero, simulando datos nunca antes vistos.


## 3. Investigar la calidad de diferentes modelos

Probaremos tres algoritmos diferentes, ajustando sus hiperparámetros para encontrar el mejor rendimiento en el conjunto de validación.

### 3.1. Árbol de Decisión (Decision Tree)

Variaremos la profundidad del árbol (max_depth) de 1 a 10.

In [7]:

print("Resultados Árbol de Decisión:")
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_val)
    acc = accuracy_score(target_val, predictions)
    
    print(f"max_depth = {depth} : {acc:.4f}")
    
    if acc > best_dt_acc:
        best_dt_acc = acc
        best_dt_model = model
        best_dt_depth = depth

print(f"\nMejor exactitud Árbol: {best_dt_acc:.4f} con profundidad {best_dt_depth}")

Resultados Árbol de Decisión:
max_depth = 1 : 0.7387
max_depth = 2 : 0.7574
max_depth = 3 : 0.7652
max_depth = 4 : 0.7636
max_depth = 5 : 0.7589
max_depth = 6 : 0.7574
max_depth = 7 : 0.7745
max_depth = 8 : 0.7667
max_depth = 9 : 0.7621
max_depth = 10 : 0.7714

Mejor exactitud Árbol: 0.7745 con profundidad 7


### 3.2. Bosque Aleatorio (Random Forest)

Probaremos combinaciones de número de estimadores (n_estimators) y profundidad (max_depth).

In [8]:
best_rf_model = None
best_rf_acc = 0
best_rf_est = 0
best_rf_depth = 0

print("Buscando mejores hiperparámetros para Random Forest...")

for est in range(10, 51, 10): # De 10 a 50 árboles
    for depth in range(1, 11): # Profundidad de 1 a 10
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        acc = model.score(features_val, target_val)
        
        if acc > best_rf_acc:
            best_rf_acc = acc
            best_rf_model = model
            best_rf_est = est
            best_rf_depth = depth

print(f"Mejor exactitud Random Forest: {best_rf_acc:.4f}")
print(f"Hiperparámetros: n_estimators={best_rf_est}, max_depth={best_rf_depth}")

Buscando mejores hiperparámetros para Random Forest...
Mejor exactitud Random Forest: 0.7978
Hiperparámetros: n_estimators=50, max_depth=10


### 3.3. Regresión Logística

Este modelo sirve como línea base de un algoritmo lineal.

In [9]:
lr_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
lr_model.fit(features_train, target_train)
lr_acc = lr_model.score(features_val, target_val)

print(f"Exactitud Regresión Logística: {lr_acc:.4f}")

Exactitud Regresión Logística: 0.7263


**Conclusion**

Durante la fase de experimentación con los datos de validación, encontramos comportamientos distintos:

**Árbol de Decisión:}** Tuvo un buen desempeño con una profundidad media *(alrededor de 7)*, alcanzando una exactitud del *77.4%*. Profundidades mayores no mejoraron el resultado, indicando sobreajuste.

**Regresión Logística:** Fue el modelo con menor rendimiento *(72.6%)*. Esto sugiere que la relación entre las características (llamadas, minutos, mensajes, datos) y el plan elegido no es linealmente separable de manera simple.

**Bosque Aleatorio:** Fue el modelo más robusto. Al combinar múltiples árboles *(50 estimadores)* con una profundidad controlada *(10)*, logró capturar mejor la complejidad de los datos, alcanzando la mayor exactitud en validación *(79.8%)*.

## 4. Comprobar la calidad del modelo

Seleccionamos el mejor modelo encontrado en la fase anterior y lo evaluamos con el conjunto de prueba. Esto nos dará una estimación no sesgada del rendimiento del modelo en el mundo real.

In [10]:
# Selección automática del mejor modelo
if best_rf_acc > best_dt_acc and best_rf_acc > lr_acc:
    final_model = best_rf_model
    model_name = "Bosque Aleatorio"
elif best_dt_acc > best_rf_acc and best_dt_acc > lr_acc:
    final_model = best_dt_model
    model_name = "Árbol de Decisión"
else:
    final_model = lr_model
    model_name = "Regresión Logística"

print(f"Modelo seleccionado: {model_name}")

# Evaluación final
test_accuracy = final_model.score(features_test, target_test)
print(f"Exactitud en conjunto de prueba: {test_accuracy:.4f}")

if test_accuracy >= 0.75:
    print("¡Objetivo cumplido! La exactitud es superior a 0.75.")
else:
    print("El modelo no alcanzó el umbral de 0.75.")

Modelo seleccionado: Bosque Aleatorio
Exactitud en conjunto de prueba: 0.7994
¡Objetivo cumplido! La exactitud es superior a 0.75.


**Conclusion**

El modelo seleccionado (**Bosque Aleatorio**) demostró ser consistente. Al someterlo al conjunto de prueba (*datos totalmente desconocidos para el modelo*), obtuvo una exactitud del **79.9%**. 

Este resultado supera con éxito el umbral establecido de **0.75**, validando que el modelo es apto para la tarea de recomendación de planes.

## 5. Prueba de cordura (Sanity Check)

Para verificar que nuestro modelo funciona mejor que el azar o que una estrategia simple, lo comparamos con un "Dummy Classifier" que siempre predice la clase más frecuente (en este caso, 'Smart' o 0).

In [11]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
dummy_acc = dummy_clf.score(features_test, target_test)

print(f"Exactitud Modelo Tonto (Dummy): {dummy_acc:.4f}")
print(f"Exactitud Nuestro Modelo:       {test_accuracy:.4f}")

if test_accuracy > dummy_acc:
    print("Prueba de cordura PASADA: El modelo es mejor que una predicción trivial.")
else:
    print("Prueba de cordura FALLIDA: El modelo no aporta valor sobre la clase mayoritaria.")

Exactitud Modelo Tonto (Dummy): 0.6952
Exactitud Nuestro Modelo:       0.7994
Prueba de cordura PASADA: El modelo es mejor que una predicción trivial.


**Conclusion**

La prueba de cordura comparó nuestro modelo contra un modelo *"tonto"* que siempre predice el plan más común (*Smart*).

**Modelo Tonto:** ~69.5% de exactitud.

**Nuestro Modelo:** ~79.9% de exactitud. La diferencia de más de 10 puntos porcentuales confirma que el modelo ha aprendido patrones reales en el comportamiento de los usuarios y no está simplemente aprovechando el desequilibrio estadístico de las clases.

# Conclusión General del Proyecto

El objetivo de desarrollar un modelo capaz de recomendar los planes Smart o Ultra a los clientes de Megaline se ha cumplido satisfactoriamente.

A través de un proceso riguroso de división de datos y ajuste de hiperparámetros, identificamos que el algoritmo de Bosque Aleatorio (Random Forest) es la herramienta más eficaz para este problema, superando a opciones más simples como la Regresión Logística o un solo Árbol de Decisión.

El modelo final alcanza una exactitud cercana al **80%**, lo que significa que de cada *10 recomendaciones que haga el sistema, **8** serán las correctas para el perfil de consumo del usuario*. Esto permitirá a Megaline dirigir sus campañas de marketing de manera más eficiente, asegurando que los clientes reciban ofertas que realmente se ajusten a sus necesidades de llamadas y datos móviles.